Here, the performance of the lookup tables optimized in the "example_F_lookup_training_test.ipynb" on state stabilization for longer time durations can be tested.

In [1]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from helpers import (
    transport_unitary,
    initialize_chain_of_zeros,
    partial_trace,
    generate_random_discrete_state,
    generate_random_bloch_state,
    test_implementations,
)
import numpy as np
from feedback_grape.utils.fidelity import fidelity
from feedback_grape.utils.operators import sigmam
from library.utils.qubit_chain_1D import embed
from feedback_grape.utils.solver import mesolve
from tqdm import tqdm
import jax
import jax.numpy as jnp

test_implementations()

In [2]:
# Physical parameters
n = 3 # number of qubits per chain (>= 3)
N_chains = 3 # Number of parallel chains to simulate
gamma = 0.25 # Decay constant
evaluation_time_steps = 50 # Number of time steps for evaluation
batch_size = 16 # Number of random states to evaluate in parallel

generate_random_state = generate_random_discrete_state
# Path to save results
results_path = "./custom_strategies/baseline_fidelities.npz"

In [ ]:
base_dim = 2**N_chains
T_half = transport_unitary(0.5, n, N_chains)
c_ops = [sum([gamma * embed(sigmam(), idx, N_chains*n) for idx in range(N_chains*n)])]

# Evolve all basis states and compute fidelities
keys = jax.random.split(jax.random.PRNGKey(42), batch_size)
states = jnp.array([generate_random_state(key, N_chains, noise_level = 0.0) for key in keys])
target_states = jnp.array([generate_random_state(key, N_chains, noise_level=0.0) for key in keys])

fidelities_each = np.zeros((len(states), evaluation_time_steps+1))
for i, (state, target_state) in enumerate(zip(states, target_states)):
    fidelities_each[i, 0] = fidelity(
        C_target=target_state,
        U_final=state,
        evo_type="density",
    )

def propagate_single_timestep(rho, rho_target):
    tmp = initialize_chain_of_zeros(rho, n=n, N_chains=N_chains)
    tmp = T_half @ tmp @ T_half.conj().T

    tmp = mesolve(
        jump_ops=c_ops,
        rho0=tmp,
    )

    tmp = T_half @ tmp @ T_half.conj().T
    tmp = partial_trace(tmp, sys_A_dim=base_dim**(n-1), sys_B_dim=base_dim)

    fid = fidelity(C_target=rho_target, U_final=tmp, evo_type="density")

    return tmp, fid

propagate_single_timestep_vmap = jax.vmap(jax.jit(propagate_single_timestep))

for i in tqdm(range(evaluation_time_steps)):
    states, fid = propagate_single_timestep_vmap(states, target_states)
    fidelities_each[:, i+1] = fid

    #for j, rho in enumerate(states):
    #    assert np.all(np.isclose(rho, rho.conj().T)), "State is not Hermitian"
    #    assert np.isclose(np.trace(rho).real, 1.0), "State is not normalized"
    #    assert np.all(np.linalg.eigvalsh(rho) >= -1e-10), "State is not positive semidefinite"

fidelities_each = fidelities_each.mean(axis=0) # Average over all basis states

# Reshape back to matrix form
np.savez(results_path, fidelities=fidelities_each)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
fidelities_each

array([1.00000001, 0.95747696, 0.91803349, 0.88152234, 0.84779079,
       0.81668348, 0.78804479, 0.76172073, 0.73756043, 0.71541728,
       0.69514984, 0.67662247, 0.65970576, 0.64427684, 0.63021952,
       0.61742435, 0.60578857, 0.59521599, 0.58561682, 0.5769075 ,
       0.56901042, 0.56185366, 0.55537075, 0.54950034, 0.54418594,
       0.53937564, 0.53502178, 0.53108073, 0.52751258, 0.52428088,
       0.52135241, 0.51869695, 0.51628699, 0.5140976 , 0.51210619,
       0.51029228, 0.5086374 , 0.50712487, 0.50573966, 0.50446826,
       0.50329851, 0.50221951, 0.5012215 , 0.50029576, 0.49943446,
       0.49863068, 0.4978782 , 0.49717153, 0.49650577, 0.49587659,
       0.49528017])